# Abstract


In [25]:
!pip install numpy pandas datetime timedelta yfinance yahoo_fin yoptions optionlab freeoptionschain eod
# from IPython.core.display import clear_output
# clear_output()

## 1. Data Collection & Technical Analysis 

In [29]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader.data as pdr
import datetime

### 1.1 Data Sources

In [30]:
# Options Data Sources
# import yahoo_fin as yf 
import yoptions as yo
import optionlab as ol
# import freeoptionschain as foc

# Premium Options Data
"""
# EOD for real time or 30 years of data (https://eodhd.com/) or (https://pypi.org/project/eod/)
"""

'\n# EOD for real time or 30 years of data (https://eodhd.com/) or (https://pypi.org/project/eod/)\n'

In [31]:
# Define the time period
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime.now()  # This will set the end date to today's date

# Ensure to use the date component only if necessary
end = end.date()  # Converts datetime object to a date object, removing time component if not needed

In [32]:
# Function to fetch option Greeks for given strikes
def get_option_greeks_for_strikes(ticker, strikes):
    # Get the options data for the given ticker
    stock = yf.Ticker(ticker)
    options_data = {}

    # Iterate through expiration dates and strikes
    for expiration in stock.options:
        opt = stock.option_chain(expiration)
        all_options = pd.concat([opt.calls, opt.puts])
        selected_options = all_options[all_options['strike'].isin(strikes)]
        
        # Fetch the Greeks for the selected options
        for index, row in selected_options.iterrows():
            greeks = (row['delta'], row['gamma'], row['theta'], row['vega'], row['rho'])
            strike = row['strike']
            options_data[strike] = greeks

    return options_data

def get_daily_woodies_pivots_with_bollinger(ticker, start_date, end_date, window=20, no_of_stds=2, strike_increment=1):
    # Fetch historical daily data
    data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    
    # Calculate Bollinger Bands
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['Bollinger High'] = rolling_mean + (rolling_std * no_of_stds)
    data['Bollinger Low'] = rolling_mean - (rolling_std * no_of_stds)
    

    # Calculate Woodie's pivot points
    data['Pivot Point'] = (data['High'] + data['Low'] + 2 * data['Close']) / 4
    data['R1'] = 2 * data['Pivot Point'] - data['Low']
    data['S1'] = 2 * data['Pivot Point'] - data['High']
    data['R2'] = data['Pivot Point'] + (data['High'] - data['Low'])     
    data['S2'] = data['Pivot Point'] - (data['High'] - data['Low'])
    data['R3'] = data['Pivot Point'] + 2 * (data['High'] - data['Low'])
    data['S3'] = data['Pivot Point'] - 2 * (data['High'] - data['Low']) 
    data['R4'] = data['Pivot Point'] + 3 * (data['High'] - data['Low'])

    # Round Bollinger Bands and pivot points to the nearest strike
    rounding_columns = ['Bollinger High', 'Bollinger Low', 'Pivot Point', 'R1', 'S1', 'R2', 'S2', 'R3', 'S3', 'R4']
    for column in rounding_columns:
        data[column + '_Strike'] = np.round(data[column] / strike_increment) * strike_increment

    # Spreads between the pivot points
    data['Bearcall_Spread_width'] = data['R1_Strike'] - data['Pivot Point_Strike']
    data['Bullput_Spread_width'] = data['Pivot Point_Strike'] - data['S1_Strike']
    
    data['Bearcall_Spread_width_2'] = data['R2_Strike'] - data['Pivot Point_Strike']
    data['Bullput_Spread_width_2'] = data['Pivot Point_Strike'] - data['S2_Strike']
    
    data['Bearcall_Spread_width_3'] = data['R3_Strike'] - data['Pivot Point_Strike']
    data['Bullput_Spread_width_3'] = data['Pivot Point_Strike'] - data['S3_Strike']

    data['BB_Spread'] = data['Bollinger High_Strike'] - data['Bollinger Low_Strike']

    # Percentage change for the spread
    # data['Bearcall_Spread_Percentage'] = data['Bearcall_Spread_width'] / data['Close']
    # data['Bullput_Spread_Percentage'] = data['Bullput_SBullput_Spread_widthpread_'] / data['Close']
    
    return data

# Define the ticker symbol
Strikes = get_daily_woodies_pivots_with_bollinger('AAPL', start, end) 
print(Strikes.tail(10))

c:\ProgramData\anaconda3\envs\jupyter-ai\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-05-28  191.509995  193.000000  189.100006  189.990005  189.990005   
2024-05-29  189.610001  192.250000  189.509995  190.289993  190.289993   
2024-05-30  190.759995  192.179993  190.630005  191.289993  191.289993   
2024-05-31  191.440002  192.570007  189.910004  192.250000  192.250000   
2024-06-03  192.899994  194.990005  192.520004  194.029999  194.029999   
2024-06-04  194.639999  195.320007  193.029999  194.350006  194.350006   
2024-06-05  195.399994  196.899994  194.869995  195.869995  195.869995   
2024-06-06  195.690002  196.500000  194.169998  194.479996  194.479996   
2024-06-07  194.649994  196.940002  194.139999  196.889999  196.889999   
2024-06-10  196.899994  197.300003  192.149994  193.119995  193.119995   

              Volume  Bollinger High  Bollinger Low  Pivot Point          R1  \
Date                           

In [11]:
index_funds = ['SPY', 'QQQ', 'DAX']
leveraged_etfs = ['TQQQ', 'UMDD', 'UDOW', 'SOXL', 'NVDL', 'TSLL']
monthly_dividend_reits_etfs = ['O', 'AGNC', 'CSHI', 'JEPI', 'NUSI']
quarterly_dividend_stocks = [
    'SPYD', 'MSFT', 'INTC', 'F', 'CSCO', 'BAC', 'PFE', 'BX', 'MO', 
    'DOW', 'WMT', 'T', 'KMB', 'SWK', 'IBM', 'PEP', 'KO', 'JNJ'
]
hedging = ['VIX', 'UVXY', 'SPXS' ]

In [1]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

# Download options data for SPY
spy = yf.Ticker("SPY")
options = spy.option_chain(date=date.today() + timedelta(days=0))

# Convert options data to dataframes
calls = pd.DataFrame(options.calls)
puts = pd.DataFrame(options.puts)

# Filter options data to only include options with 0 days to expiration
zero_day_calls = calls[calls['daysToExpiration'] == 0]
zero_day_puts = puts[puts['daysToExpiration'] == 0]

# Print the filtered options data
print(zero_day_calls)
print(zero_day_puts)


ValueError: Expiration `2024-06-12` cannot be found. Available expirations are: [2024-06-12, 2024-06-13, 2024-06-14, 2024-06-17, 2024-06-18, 2024-06-20, 2024-06-21, 2024-06-28, 2024-07-05, 2024-07-12, 2024-07-19, 2024-07-26, 2024-07-31, 2024-08-16, 2024-08-30, 2024-09-20, 2024-09-30, 2024-10-18, 2024-10-31, 2024-11-15, 2024-11-29, 2024-12-20, 2024-12-31, 2025-01-17, 2025-01-31, 2025-03-21, 2025-03-31, 2025-06-20, 2025-09-19, 2025-12-19, 2026-01-16, 2026-06-18, 2026-12-18]

In [ ]:
from yahoo_fin import options
import pandas as pd
stock = "SPY"
pd.set_option('display.max_columns', None)
chain = options.get_options_chain(stock)
print("CALLS \n \n", chain['calls'].tail() )
print("\n \n PUTS \n \n", chain['puts'].tail()  )

CALLS 
 
          Contract Name         Last Trade Date  Strike  Last Price  Bid   Ask  \
71  SPY240221C00535000   2024-02-14 3:44PM EST   535.0        0.01  0.0  0.01   
72  SPY240221C00540000  2024-02-20 12:16PM EST   540.0        0.01  0.0  0.01   
73  SPY240221C00545000   2024-02-13 2:34PM EST   545.0        0.01  0.0  0.01   
74  SPY240221C00550000   2024-02-16 1:15PM EST   550.0        0.01  0.0  0.01   
75  SPY240221C00570000  2024-02-20 12:59PM EST   570.0        0.01  0.0  0.01   

    Change % Change Volume  Open Interest Implied Volatility  
71     0.0        -      2              5             50.00%  
72     0.0        -      9             17             53.13%  
73     0.0        -      -             10             57.81%  
74     0.0        -      1              1             62.50%  
75     0.0        -      4              4             84.38%  

 
 PUTS 
 
          Contract Name         Last Trade Date  Strike  Last Price   Bid  \
91  SPY240221P00519000  2024-02-16 1

### 1.3. Technical Indicators

### 1.3. Data Cleaning



### 1.4. Splitting Market Segments (Pre-Market, Market, After-Hours)



## 2. Data Visualization

### 2.1. Charting and Candlestick



### 2.2. Trend



### 2.3. Volume

### 2.4. Momentium

### 2.2. Volatility

## 3. Correlation Analysis
(Coefficent, Direction, Strength, Significance, Types of Correlation, Matrix)

## 4. Feature Engineering

## 5. Risk Analysis & Management
(Value at Risk (VaR))
(Conditional Value at Risk (CVaR) )
(Sharpe Ratio)
(Hedge Ratio)
(Violatility Stop based on ATR)

## 6. Backtest Portiolio